In [ ]:
# --- Imports ---------------------------------------------------------------
from datetime import date
import pandas as pd

from expedia.client import ExpediaClient
from expedia.geo_helpers import circle_polygon_geojson
from expedia.helpers.rates_report import (
    DEFAULT_OCCUPANCY,
    DEFAULT_RATE_TYPES,
    RATE_TYPE_LABELS,
    SEARCH_POINT_DENSITY,
    generate_rates_dataframe,
    export_rates_to_excel,
)

client = ExpediaClient.from_env()

# --- Inputs ----------------------------------------------------------------
LATITUDE = 52.370216      # float
LONGITUDE = 4.895168      # float
RADIUS_KM = 3             # float/int
CHECKIN = date(2025, 11, 20)   # date or "YYYY-MM-DD"
CHECKOUT = date(2025, 11, 22)  # date or "YYYY-MM-DD"

# Optional overrides
OCCUPANCY = DEFAULT_OCCUPANCY
RATE_TYPES = DEFAULT_RATE_TYPES       # e.g. ("mkt_prepay", "priv_pkg")
RATE_LABELS = RATE_TYPE_LABELS        # customise if desired
POINT_DENSITY = SEARCH_POINT_DENSITY  # 96 matches the app

polygon = circle_polygon_geojson(
    center_lat=LATITUDE,
    center_lon=LONGITUDE,
    radius_m=RADIUS_KM * 1000,
    n_points=96,
)

property_ids = client.search_geography(
    polygon,
    include="property_ids",
    supply_source="expedia",
    checkin=str(CHECKIN),
    checkout=str(CHECKOUT),
)

# --- Fetch rates -----------------------------------------------------------
# rates_df, meta = generate_rates_dataframe(
#     client,
#     latitude=LATITUDE,
#     longitude=LONGITUDE,
#     radius_km=RADIUS_KM,
#     checkin=CHECKIN,
#     checkout=CHECKOUT,
#     occupancy=OCCUPANCY,
#     rate_types=RATE_TYPES,
#     n_points=POINT_DENSITY,
#     rate_type_labels=RATE_LABELS,
# )


In [65]:
ids = 97
avail = client.fetch_availability(
    property_ids[ids], 
    str(CHECKIN), 
    str(CHECKOUT),
    2,
    "priv_pkg",
    )
details = client.fetch_property_summaries(
    [property_ids[ids]]
    )

import json
with open("data/room.json", "w") as f:
    json.dump(avail, f, indent=2)

room = next(iter(avail))
px = avail[room]["rooms"][0]["rates"][0]["occupancy_pricing"]['2']["totals"]["inclusive"]["request_currency"]["value"]
commission = avail[room]["rooms"][0]["rates"][0]["occupancy_pricing"]['2']["totals"]["marketing_fee"]["request_currency"]["value"]
print(f"price: {px}; commission: {commission}")
print(f"{round(float(commission) / float(px) * 100, 2)}% commission")


price: 650.41; commission: 51.15
7.86% commission


In [66]:
details

{'1521568': {'property_id': '1521568',
  'name': 'Hotel des Arts',
  'address': {'line_1': 'Rokin 154',
   'city': 'Amsterdam',
   'postal_code': '1012 LE',
   'country_code': 'NL',
   'obfuscation_required': False,
   'localized': {'links': {'en-US': {'method': 'GET',
      'href': 'https://api.ean.com/v3/properties/content?language=en-US&property_id=1521568&include=address&supply_source=expedia'}}}},
  'location': {'coordinates': {'latitude': 52.36797, 'longitude': 4.892591},
   'obfuscation_required': False},
  'statistics': {'52': {'id': '52',
    'name': 'Total number of rooms - 22',
    'value': '22'},
   '54': {'id': '54', 'name': 'Number of floors - 5', 'value': '5'}}}}

In [1]:
# Notebook cell 1 – imports and helpers
import pandas as pd
from pathlib import Path

from expedia.client import ExpediaClient
from expedia.event_rates_cache import update_event_rates_cache
from events.event_helpers import load_events_catalog

CACHE_PATH = Path("reports/events_with_hotels.json")
EVENT_DATA_DIRECTORIES = [Path("data"), Path("events/data")]

events_catalog = load_events_catalog(EVENT_DATA_DIRECTORIES)
print(f"{len(events_catalog)} events loaded")

client = ExpediaClient.from_env()  # assumes ENV vars or .env populated


Loaded event sources:
  - events/data/2025_events.json
  - events/data/2026_events.json
  - events/data/2027_events.json
  - events/data/2028_events.json
  - events/data/USA_events.xlsx
  - events/data/all_events.xlsx
2481 events loaded


In [4]:
from pathlib import Path
import json
import pandas as pd
import builtins

def list_cached_event_names(cache_path: Path = Path("reports/events_with_hotels.json")) -> list[str]:
    """Load the events cache and return a list of event titles."""
    if not cache_path.exists():
        return []

    try:
        data = json.loads(cache_path.read_text(encoding="utf-8"))
    except (OSError, json.JSONDecodeError):
        return []

    if isinstance(data, dict):
        data = [data]
    elif not isinstance(data, list):
        return []

    names: list[str] = []
    for entry in data:
        if isinstance(entry, dict):
            title = entry.get("title")
            if isinstance(title, builtins.str) and title.strip():
                names.append(title.strip())

    return names



def drop_events_by_name(df: pd.DataFrame, banned_names: list[str]) -> pd.DataFrame:
    """
    Return a copy of `df` with rows removed when `name` matches one of `banned_names`.
    Non-string values in the column are ignored.
    """
    if "title" not in df.columns or not banned_names:
        return df.copy()

    mask = df["title"].astype(str).isin({name for name in banned_names if isinstance(name, str)})
    return df.loc[~mask].reset_index(drop=True)


In [5]:
# USA 15 Oct
# Netherlands, Canada, Mexico, UK, 17 oct
# EU-27, Switzerland, Monaco, Norway 18 Oct
from expedia.geo_helpers import circle_polygon_geojson, to_geojson_string, haversine_distance

import pyperclip
import re
import pandas as pd


countries = ["United Arab Emirates", "UAE"
]


target_countries = countries
pattern = "|".join(re.escape(name) for name in target_countries)

events_catalog_f = events_catalog[
    events_catalog["country"]
        .astype(str)
        .str.contains(pattern, case=False, na=False)
]

from events.event_helpers import get_cached_event_map_keys

cached_keys = {key.lower() for key in get_cached_event_map_keys("reports/events_with_hotels.json")}
events_catalog_f = events_catalog_f[
    ~events_catalog_f["map_key"].astype(str).str.lower().isin(cached_keys)
]

events_catalog_f = events_catalog_f[pd.to_datetime(events_catalog_f["date_start"]) >= pd.Timestamp.today().normalize()]
events_catalog_f = events_catalog_f[
    pd.to_datetime(events_catalog_f["date_end"]) - pd.to_datetime(events_catalog_f["date_start"]) < pd.Timedelta(days=28)
]
events_catalog_f = events_catalog_f[
    pd.to_datetime(events_catalog_f["date_end"]) <= pd.Timestamp.today().normalize() + pd.Timedelta(days=500)
]

country_events = events_catalog_f[events_catalog_f["date_start"] != events_catalog_f["date_end"]].reset_index(drop=True)

len(country_events)


106

In [6]:
events_to_cache = country_events
chunk = 10
tot = len(events_to_cache)
begin = 0
for i in range(begin, tot, chunk):
    start = i
    end = min(i + chunk - 1, tot - 1)
    
    print(start, end)

    selected_events = events_to_cache.iloc[start:end]
    # Notebook cell 3 – cache the selected event search
    limit = 100
    records = update_event_rates_cache(
        selected_events,
        client,
        output_path=CACHE_PATH,
        limit=limit,
        show_progress=True,          # optional: prints progress per event
        default_radius_km=6.0,       # override if your catalog omits radius_km
        retry_kwargs={
            "max_attempts": 5,
            "initial_backoff_seconds": 8.0,
            "backoff_multiplier": 1.5,
            "post_success_sleep_seconds": 2.0,
        },
    )

    print(f"{len(records)} total events now cached in {CACHE_PATH}")


0 9


KeyboardInterrupt: 

In [ ]:
# find the nearest hotel to an event. Get price and discount
ids = 10
avail = client.fetch_availability(
    property_ids[ids], 
    str(CHECKIN), 
    str(CHECKOUT),
    2,
    "priv_pkg",
    )

details = client.fetch_property_summaries(
    [property_ids[ids]]
    )

# import json
# with open("data/room.json", "w") as f:
#     json.dump(avail, f, indent=2)

room = next(iter(avail))
px = avail[room]["rooms"][0]["rates"][0]["occupancy_pricing"]['2']["totals"]["inclusive"]["request_currency"]["value"]
commission = avail[room]["rooms"][0]["rates"][0]["occupancy_pricing"]['2']["totals"]["marketing_fee"]["request_currency"]["value"]
# print(f"price: {px}; commission: {commission}")
# print(f"{round(float(commission) / float(px) * 100, 2)}% commission")

df = pd.DataFrame()

price: 650.80; commission: 51.18
7.86% commission


In [ ]:
event = events_catalog.iloc[0]
generate_rates_dataframe(client, event[])

In [69]:
event = events_catalog.iloc[0]
event

title                                             Healthcare Walks
city                                                   Kansas City
country                                              United States
latitude                                                 39.098253
longitude                                               -94.587488
venue_id                                                    483369
map_key                                                      MTZDN
date_start                                     2025-12-02 00:00:00
date_end                                       2025-12-03 00:00:00
source                                events/data/2025_events.json
display_label    Healthcare Walks • Kansas City • 02 Dec 2025 →...
search_blob      healthcare walks kansas city united states mtz...
Name: 0, dtype: object

In [71]:
details = client.fetch_property_summaries(
    ['8799']
    )
details

{'8799': {'property_id': '8799',
  'name': 'Leonardo Hotel Amsterdam City Center',
  'address': {'line_1': 'Tesselschadestraat 23',
   'city': 'Amsterdam',
   'postal_code': '1054 ET',
   'country_code': 'NL',
   'obfuscation_required': False,
   'localized': {'links': {'en-US': {'method': 'GET',
      'href': 'https://api.ean.com/v3/properties/content?language=en-US&property_id=8799&include=address&supply_source=expedia'}}}},
  'location': {'coordinates': {'latitude': 52.3622, 'longitude': 4.87843},
   'obfuscation_required': False},
  'statistics': {'52': {'id': '52',
    'name': 'Total number of rooms - 103',
    'value': '103'}}}}